# Efficient Yelp API Calls (Core)

**Goal:** 

Use the Yelp API to search your favorite city for a cuisine type of your choice.

Extract all of the results from your search and compile them into one dataframe using a for loop.

In [1]:
# imports

import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# import json to read yelp api credentials
import json

# with open: yelp api credentials (save as variable)
with open('/Users/yang0108/.secret/yelp_api.json') as f:
    login = json.load(f)

login.keys()

dict_keys(['client-id', 'api-key'])

In [3]:
# import yelpapi class
from yelpapi import YelpAPI

# create instance with api key
yelp_api = YelpAPI(login['api-key'], timeout_s = 5.0)
yelp_api

In [5]:
# define location and term
LOCATION = 'Seattle, WA'
TERM = 'Korean'

In [4]:
# specify JSON_FILE to save results to
JSON_FILE = 'Data/results_in_progress_Seattle_Korean.json'
JSON_FILE

'Data/results_in_progress_Seattle_Korean.json'

In [ ]:
# define function to create and save an empty JSON file
def create_json_file(JSON_FILE, delete_if_exists = False):
    
    # check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    # if it DOES exist:
    if file_exists == True:
        
        # check if user wants to delete if exists
        if delete_if_exists == True:
            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file.")
            
            # delete previous file and confirm it no longer exists
            os.remove(JSON_FILE)
            
            # recursive call to function after old file is deleted
            create_json_file(JSON_FILE, delete_if_exists = False)
    
    # if it does NOT exist:
    else:
        
        # inform user and save empty list
        print(f"[!] {JSON_FILE} not found. Saving empty list to new file.")
        
        # create any needed folders specified in the JSON_FILE path
        # get the folder name
        folder = os.path.dirname(JSON_FILE)
        
        # if JSON_FILE path included a folder:
        if len(folder) > 0:
            
            # create the folder
            os.makedirs(folder, exist_ok = True)
            
            # save empty list to start the json file
            with open(JSON_FILE, 'w') as f:
                json.dump([], f)

In [ ]:
# use search_query method to perform api call
search_results_0 = yelp_api.search_query(location = LOCATION,
                                         term = TERM)

# check
print(type(search_results_0))
search_results_0.keys()